In [131]:
from jedi.inference.utils import to_list

# read it in to inspect it
with open ('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# print("the length of the array is: ", len(text))
# print(text[:1000])


In [132]:
#all the characters that occur in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [133]:
#create a mapping for characters to integers
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# print(encode("hii there"))
# print(decode(encode("hii there")))


In [134]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch  # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])  # the 1000 characters we looked at earlier will to the GPT look like this

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]


torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [135]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [136]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x,y)

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor([47, 56, 57, 58,  1, 15, 47, 58])
when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [137]:
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # if for example len is 30 it generates numbers in [0, 22); [0,21]
    x = torch.stack([data[i:i+block_size] for i in ix]) # if 21 is generated [21: 29) in other words [21: 28]
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # if 21 is generated [22: 30) in other words [22: 29]
    return x, y
# 
xb, yb = get_batch('train')
# # print('inputs  :') 
# # print(xb.shape)
# # print(xb)
# # print('targets:')
# # print(yb.shape)
# # print(yb)

print('----')
for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")


----
when input is [56] the target: 57
when input is [56, 57] the target: 43
when input is [56, 57, 43] the target: 57
when input is [56, 57, 43, 57] the target: 1
when input is [56, 57, 43, 57, 1] the target: 47
when input is [56, 57, 43, 57, 1, 47] the target: 52
when input is [56, 57, 43, 57, 1, 47, 52] the target: 1
when input is [56, 57, 43, 57, 1, 47, 52, 1] the target: 46
when input is [43] the target: 56
when input is [43, 56] the target: 63
when input is [43, 56, 63] the target: 1
when input is [43, 56, 63, 1] the target: 46
when input is [43, 56, 63, 1, 46] the target: 39
when input is [43, 56, 63, 1, 46, 39] the target: 42
when input is [43, 56, 63, 1, 46, 39, 42] the target: 1
when input is [43, 56, 63, 1, 46, 39, 42, 1] the target: 57
when input is [42] the target: 43
when input is [42, 43] the target: 0
when input is [42, 43, 0] the target: 13
when input is [42, 43, 0, 13] the target: 1
when input is [42, 43, 0, 13, 1] the target: 51
when input is [42, 43, 0, 13, 1, 51] t

In [138]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) 

    def forward(self, idx, targets=None ): # This function receives B by T tensor of elements 
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (B,T,C)
    
        if targets is None:
            loss = None
        else: #when no target is defined this else is not run therefore the dimensionality of logits will not change
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets  = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is B by T array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:] # now it becomes B by C since we only plucked out the last row of each batch 
            probs = F.softmax(logits, dim=-1) # also B by C; one probability per each class for all batches
            idx_next = torch.multinomial(probs, num_samples=1) #one suggestion to be added to the last element of each batch
            idx = torch.cat((idx, idx_next), dim=-1)
        return idx
            
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
# [0] is because the nature of the m.generate output is still a 2D tensor and the elements in the first row are called using the row index and the column index, so if we want to have the whole row together we need to use [0]
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8513, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [139]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [149]:
batch_size = 32
for _ in range(100000):
    # here we are sampling data
    xb, yb = get_batch('train')
    
    # here we evaluate the loss and optimize using gradients
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
        
print(loss.item())
    
    

2.385108232498169


In [152]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long),max_new_tokens=1000)[0].tolist()))





Haye oathawee
Whathiesovenn n wio.
LABom TES:
HAntouged derisly;
INES:
PO wiowado s at lfe. wigoan l t whedanthebe winor 'd.
Card as ooracollt, thes
se wo mane y wond.
INGRI s GELit. forofedeat s frgss,
K:
Hal t t; fo n.
'sourmonem o wh wef thoy au, her land ges urndom howhugary wassucavom? The me d,
RG adoueentand ourk fre Ifot su we t.
NTENGeh, trde assenjoche t bos, swar heonas thintousear!
HELem:
Gotheret
LELUS:
TE kig f s CHo shene posed ather anjor'st prs d I iry,
To ourd toth muric t Is s nean chonicasersie I cct bllllediap, f se,
ISOManne s,
INI ABurdighy ha---pl be dvethens oth r per. me thot:
A:

N g htint aderesed oud tilfrt au then:
Cornngs n ghonth, aiere I s,
Sis y d h GLoway cer ther. t, m'tunow, gou'diny ABEDate cut, fis
ad k to,
An's he--phepthist thof linghe, s wan, th t th ol e pat Lar the thounfra w, ft thethe.
Heronk'e st hag there t ims he.
Bueeeret s this, suile for t ild.
Alle dro ddif t
ERKIZE t I pe corsord'th grmemy,
PEDYCARDUCLAPons pavepothon pllanceit w